
# Projet 2

In [ ]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [ ]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [ ]:
geo['CP Ville'] = geo['Code Postal'] + ' ' + geo['Commune']

In [ ]:
# fonction recherche de ville
def ville(lat, long):
    dist = np.sqrt(((geo['Latitude']-lat)**2 + (geo['Longitude']-long)**2))
    return geo.loc[dist.idxmin(), 'CP Ville']

In [ ]:
# fonction recherche de ville
def ville(lat, long):
    dist2 = ((geo['Latitude']-lat) ** 2 + (geo['Longitude']-long) ** 2)
    return geo.loc[dist2.idxmin(), 'CP Ville']

In [ ]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

In [ ]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [ ]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

In [ ]:
# à partir de coordonnées GPS précises
ville(dms2dec(, , ), dms2dec(, , ))

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [ ]:
import json

geo_shape = geo['geo_shape'].apply(json.loads)
geo_shape

In [ ]:
geo_shape.apply(lambda x: x['type']).value_counts()

In [ ]:
geo_shape.apply(lambda x: x['coordinates']).apply(len).value_counts()

In [ ]:
geo.loc[geo_shape.apply(lambda x: x['coordinates']).apply(len).idxmax(), 'CP Ville']

In [ ]:
geo.loc[geo_shape.apply(lambda x: x['coordinates']).apply(len)==3, 'CP Ville']

In [ ]:
geo.loc[geo_shape.apply(lambda x: x['type']=='Polygon')
        & (geo_shape.apply(lambda x: x['coordinates']).apply(len)==2), 'CP Ville']

In [ ]:
from shapely.geometry import Polygon

def test_enclave(coordinates):
    if len(coordinates) != 2:
        return False
    return Polygon(coordinates[0]).contains(Polygon(coordinates[1]))

selection = geo_shape.loc[geo_shape.apply(lambda x: x['type']=='Polygon') &
                          (geo_shape.apply(lambda x: x['coordinates']).apply(len)==2)]
selection.apply(lambda x: test_enclave(x['coordinates'])).all()